**this is an example notebook for querying from the Opensantions API**

*Proceeding:* 

1. Query the individual sanctions lists from the website and write a loop to access all the days from July 2021 onward 

2. Parse into a dataframe, that is filtered and cleaned 

3. Write a function that matches a day with the previous day and flags the new additions or deletions. Parse those as a new column indicating removal or addition date.  

4. Merge all dataframes for all lists (UK, EU and US) together and aggregate onto a monthly level 

5. Create a separate dataframe for all designations concerning Russian entities 

 
datasets of interest:

us_ofac_sdn

us_ofac_cons

us_bis_denied


eu_fsf

eu_sanctions_map


loop though versions of the json not the API
ok - I just confirmed, the API doesn't have historical data - but all the historical data is available online still. You would have to request the file you want to check for each date, download the file, and then read through it and filter for what you're looking for
14:38 Uhr
e.g. the latest OFAC file is https://data.opensanctions.org/datasets/20240115/us_ofac_cons/entities.ftm.json
14:39 Uhr
you can then loop back on the date in the URL - the date is in the format YYYYMMDD
14:39 Uhr
e.g.  https://data.opensanctions.org/datasets/20240114/us_ofac_cons/entities.ftm.json and so on

In [1]:
#import packages
import requests
import json
import pandas as pd
import numpy as np
import os

**start with the SDN list**

In [2]:
#most up to date datafile in json:
#https://data.opensanctions.org/datasets/20240114/us_ofac_cons/entities.ftm.json 
#now I need to create a loop to access the datafiles from Jul 2021 to Dec 2023
#first create a list of datetimes to use in the for loop
date_list = pd.date_range(start='20210507',end='20231231',freq='D').strftime('%Y%m%d')
date_list

Index(['20210507', '20210508', '20210509', '20210510', '20210511', '20210512',
       '20210513', '20210514', '20210515', '20210516',
       ...
       '20231222', '20231223', '20231224', '20231225', '20231226', '20231227',
       '20231228', '20231229', '20231230', '20231231'],
      dtype='object', length=969)

In [3]:
import requests, json
websites = []

for i in date_list:
    test = 'https://data.opensanctions.org/datasets/'+(i)+'/us_ofac_sdn/entities.ftm.json'
    websites.append(test)
print(websites)

['https://data.opensanctions.org/datasets/20210507/us_ofac_sdn/entities.ftm.json', 'https://data.opensanctions.org/datasets/20210508/us_ofac_sdn/entities.ftm.json', 'https://data.opensanctions.org/datasets/20210509/us_ofac_sdn/entities.ftm.json', 'https://data.opensanctions.org/datasets/20210510/us_ofac_sdn/entities.ftm.json', 'https://data.opensanctions.org/datasets/20210511/us_ofac_sdn/entities.ftm.json', 'https://data.opensanctions.org/datasets/20210512/us_ofac_sdn/entities.ftm.json', 'https://data.opensanctions.org/datasets/20210513/us_ofac_sdn/entities.ftm.json', 'https://data.opensanctions.org/datasets/20210514/us_ofac_sdn/entities.ftm.json', 'https://data.opensanctions.org/datasets/20210515/us_ofac_sdn/entities.ftm.json', 'https://data.opensanctions.org/datasets/20210516/us_ofac_sdn/entities.ftm.json', 'https://data.opensanctions.org/datasets/20210517/us_ofac_sdn/entities.ftm.json', 'https://data.opensanctions.org/datasets/20210518/us_ofac_sdn/entities.ftm.json', 'https://data.o

In [37]:
from urllib.request import urlopen
contents_list=[]
json_data=[]
for site in websites:
    websites = site
    r = urlopen(website)
    json_data = r.read().decode('utf-8', 'replace')
    contents_list.append(json_data)

display(contents_list)

['{"caption": "MIDDLE BAZZAR, RAMPUR BUSHAHR, DISTT. SHIMLA (H.P.) 172 001 -", "datasets": ["us_bis_denied"], "first_seen": "2021-09-27T09:09:30", "id": "addr-00a2f59121fb7d67483541de07479c63213b53ef", "last_seen": "2022-06-21T18:15:07", "properties": {"city": ["DISTT. SHIMLA (H.P.) 172 001"], "country": ["in"], "full": ["MIDDLE BAZZAR, RAMPUR BUSHAHR, DISTT. SHIMLA (H.P.) 172 001 -"], "street": ["MIDDLE BAZZAR, RAMPUR BUSHAHR"]}, "referents": [], "schema": "Address", "target": false}\n{"caption": "INMATE NUMBER: 73792-112, FCI SAFFORD, P.O. BOX 9000, 85548 SAFFORD", "datasets": ["us_bis_denied"], "first_seen": "2021-09-27T09:09:30", "id": "addr-00fd8d958773d626b9531e31fb751921bcfd76ca", "last_seen": "2022-06-21T18:15:07", "properties": {"city": ["SAFFORD"], "full": ["INMATE NUMBER: 73792-112, FCI SAFFORD, P.O. BOX 9000, 85548 SAFFORD"], "postalCode": ["85548"], "region": ["AZ"], "street": ["INMATE NUMBER: 73792-112, FCI SAFFORD, P.O. BOX 9000"]}, "referents": [], "schema": "Address", 

In [42]:
#merge and parse into one df
#from json import json_normalize
#df = pd.json_normalize(contents_list)
#df
data_ofac = pd.DataFrame(contents_list)
data_ofac

,0
0,"{""caption"": ""MIDDLE BAZZAR, RAMPUR BUSHAHR, DI..."


In [ ]:
#filter and clean up

repeat with EU and other lists

**OFAC Consolidated list**

In [45]:
websites = []

for i in date_list:
    test = 'https://data.opensanctions.org/datasets/'+(i)+'/us_ofac_cons/entities.ftm.json'
    websites.append(test)
print(websites)

['https://data.opensanctions.org/datasets/20210507/us_ofac_cons/entities.ftm.json', 'https://data.opensanctions.org/datasets/20210508/us_ofac_cons/entities.ftm.json', 'https://data.opensanctions.org/datasets/20210509/us_ofac_cons/entities.ftm.json', 'https://data.opensanctions.org/datasets/20210510/us_ofac_cons/entities.ftm.json', 'https://data.opensanctions.org/datasets/20210511/us_ofac_cons/entities.ftm.json', 'https://data.opensanctions.org/datasets/20210512/us_ofac_cons/entities.ftm.json', 'https://data.opensanctions.org/datasets/20210513/us_ofac_cons/entities.ftm.json', 'https://data.opensanctions.org/datasets/20210514/us_ofac_cons/entities.ftm.json', 'https://data.opensanctions.org/datasets/20210515/us_ofac_cons/entities.ftm.json', 'https://data.opensanctions.org/datasets/20210516/us_ofac_cons/entities.ftm.json', 'https://data.opensanctions.org/datasets/20210517/us_ofac_cons/entities.ftm.json', 'https://data.opensanctions.org/datasets/20210518/us_ofac_cons/entities.ftm.json', 'ht

In [46]:
from urllib.request import urlopen
contents_list=[]
json_data=[]
for site in websites:
    websites = site
    r = urlopen(website)
    json_data = r.read().decode('utf-8', 'replace')
    contents_list.append(json_data)

display(contents_list)

**US BIS List**

In [47]:
websites = []

for i in date_list:
    test = 'https://data.opensanctions.org/datasets/'+(i)+'/us_bis_denied/entities.ftm.json'
    websites.append(test)
print(websites)

['https://data.opensanctions.org/datasets/20210507/us_bis_denied/entities.ftm.json', 'https://data.opensanctions.org/datasets/20210508/us_bis_denied/entities.ftm.json', 'https://data.opensanctions.org/datasets/20210509/us_bis_denied/entities.ftm.json', 'https://data.opensanctions.org/datasets/20210510/us_bis_denied/entities.ftm.json', 'https://data.opensanctions.org/datasets/20210511/us_bis_denied/entities.ftm.json', 'https://data.opensanctions.org/datasets/20210512/us_bis_denied/entities.ftm.json', 'https://data.opensanctions.org/datasets/20210513/us_bis_denied/entities.ftm.json', 'https://data.opensanctions.org/datasets/20210514/us_bis_denied/entities.ftm.json', 'https://data.opensanctions.org/datasets/20210515/us_bis_denied/entities.ftm.json', 'https://data.opensanctions.org/datasets/20210516/us_bis_denied/entities.ftm.json', 'https://data.opensanctions.org/datasets/20210517/us_bis_denied/entities.ftm.json', 'https://data.opensanctions.org/datasets/20210518/us_bis_denied/entities.ft

In [ ]:
from urllib.request import urlopen
contents_list=[]
json_data=[]
for site in websites:
    websites = site
    r = urlopen(website)
    json_data = r.read().decode('utf-8', 'replace')
    contents_list.append(json_data)

display(contents_list)

**EU FSF**

In [ ]:
websites = []

for i in date_list:
    test = 'https://data.opensanctions.org/datasets/'+(i)+'/eu_fsf/entities.ftm.json'
    websites.append(test)
print(websites)

In [ ]:
from urllib.request import urlopen
contents_list=[]
json_data=[]
for site in websites:
    websites = site
    r = urlopen(website)
    json_data = r.read().decode('utf-8', 'replace')
    contents_list.append(json_data)

display(contents_list)

**EU Sanctions Map**

In [ ]:
websites = []

for i in date_list:
    test = 'https://data.opensanctions.org/datasets/'+(i)+'/eu_sanctions_map/entities.ftm.json'
    websites.append(test)
print(websites)

In [ ]:
from urllib.request import urlopen
contents_list=[]
json_data=[]
for site in websites:
    websites = site
    r = urlopen(website)
    json_data = r.read().decode('utf-8', 'replace')
    contents_list.append(json_data)

display(contents_list)

In [ ]:
eu_travel_bans